# FHI Module 7 Hands-on --Introduction


## 1. Introduction of the data

[MIMIC-II](https://physionet.org/mimic2/mimic2_clinical_overview.shtml) is a freely available database of ICU patients. To access the full database (now migrated to [MIMIC-III](https://www.nature.com/articles/sdata201635.pdf))  you must sign a data use agreement. However, there is a [demo data set](https://physionet.org/mimic2/demo/) based on 4000 deceased patients that can be used without signing any DUA [1].

### 1.1 How to Use the MIMIC-II Database
* [MIMIC-II Cookbook](https://physionet.org/mimic2/demo/MIMICIICookBook_v1.pdf)
* [MIMIC Data Dictionaries](http://physionet.incor.usp.br/physiobank/database/dictionaries/)


### 1.2 The Varieties of...Data
The data set is very rich and so is a good resource for exploring the varieties of clinical data

[database schemas at glance](https://mit-lcp.github.io/mimic-schema-spy/)

Data incluces free text notes (nursing, radiology, discharg summaries, etc.), input/output events, test results, procedure codes, diagnosis codes, etc.

### 1.3 How did we prepare this dataset

1) **Retrieved all the MD notes**    
    Because the family history is usually recorded by doctors. We want to find more *positive mentions of the target concepts* when creating this dataset. Why?   
    In this dataset, there are only two types of physician notes: 'discharge summary' and 'MD notes.' The latter only has a few and doesn't have any family history mentions. So we only retrieve discharge summaries.  
    More details of how to, can be found [Retrieve the notes from MIMIC db](advanced/Retrieve the notes from MIMIC db.ipynb)

2) **Further narrowed down the notes**    
    Used text search to identify notes that have relevant phrases like *'breast cancer', 'colon cancer', etc. (needs to be rich and complete).   
    
3) **Randomly pulled notes from the rest as negative samples**    
    Manually went over the notes to make sure that there are no mentions of our interest.
    
4) **Preprocessed the documents**  
    Replaced the PHI markups with mimic real information. For example, replaced '[\*\* Known patient lastname \*\*]' in an original document with a random name 'Adams'.  
    
5) **Annotated the dataset**    
    Annotated the dataset using [Brat](http://brat.nlplab.org/).
    
6) **Packaged up**    
    Splitted the dataset through group sampling into training (60%) and testing (40%). Zipped them into two zip files. More details of how to do it, can be found [SplitDataset.ipynb](advanced/SplitDataset.ipynb)




## 2. Explore the data

Let's take a look at the annotated data, see how it looks like.

In [1]:
# import packages that we will need
from nlp_pneumonia_utils import read_doc_annotations
from nlp_pneumonia_utils import DocumentClassifier
from nlp_pneumonia_utils import list_errors
from visual import snippets_markup
from visual import view_pycontext_output
from visual import display_doc_text
# packages for interaction
from IPython.display import display, HTML
import ipywidgets

### 2.1 Load our training set

In [2]:
# first we need to tell which type is considered for positive document
pos_doc_type='FAM_BREAST_CA_DOC'

annotated_doc_map = read_doc_annotations(archive_file='data/bc_train.zip', pos_type=pos_doc_type)

# let's also use a simple list of documents as well as this map
annotated_docs = list(annotated_doc_map.values())

print('Total Annotated Documents : {0}'.format(len(annotated_docs)))

Reading annotations from file : data/bc_train.zip
Opening local file : data/bc_train.zip
Total Annotated Documents : 60


### 2.2 See what have been annotated

In [3]:
#check notes have positive mentions

# limit the number of documents to display

total_display=15

pos_docs=dict((k, v) for k, v in annotated_doc_map.items() if  v.annotations[0].type ==pos_doc_type)

display(HTML(snippets_markup(dict(list(pos_docs.items())[:total_display]),'FAM_BREAST_CA', 900,400)))

document name,Snippets
24880,
,y is significant only for a maternal niece with breast cancer at the age of 78. Physical Exam: Vitals - 97
5834,
,"y History: Father with lung ca at 76, mother with breast ca at 67, one grandson with cystic fibrosis. Phys"
24389,
,. Family History: Family History: One sister: Breast cancer and colon cancer. One aughter: Breast cancer an
,": Breast cancer and colon cancer. One aughter: Breast cancer and colon cancer, thrombocytosis. Other daughte"
26440,
,of a stroke. She has a daughter living at 54 with breast cancer. PHYSICAL EXAMINATION ON PRESENTATION: Physica
20838,


See where the snippets came from:

In [5]:
display_doc_text(pos_docs,300)

interactive(children=(IntSlider(value=0, description='i', max=31), Output()), _dom_classes=('widget-interact',))

### 2.3 See negative documents

In [6]:
neg_docs=dict((k, v) for k, v in annotated_doc_map.items() if  v.annotations[0].type !=pos_doc_type)

# comment the line below to clear the output cell
display_doc_text(neg_docs,300)

interactive(children=(IntSlider(value=0, description='i', max=27), Output()), _dom_classes=('widget-interact',))

## References:
1. MIMIC-III, a freely accessible critical care database. Johnson AEW, Pollard TJ, Shen L, Lehman L, Feng M, Ghassemi M, Moody B, Szolovits P, Celi LA, and Mark RG. Scientific Data (2016). DOI: 10.1038/sdata.2016.35. Available at: http://www.nature.com/articles/sdata201635